# Precipitation

## Mean precip

In [ ]:
%reload_ext autoreload

%autoreload 2

import math
import os

import iris
import iris.analysis.cartography
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from ml_downscaling_emulator import UKCPDatasetMetadata
from ml_downscaling_emulator.helpers import plot_over_ts
from ml_downscaling_emulator.utils import cp_model_rotated_pole, platecarree, plot_grid
from ml_downscaling_emulator.preprocessing.select_domain import SelectDomain

In [ ]:
uk_mon_pr_ds = xr.open_mfdataset("/home/henry/cdt/summer/data/ceda/uk/2.2km/rcp85/01/pr/mon/*.nc")
uk_mon_pr_ds

In [ ]:
uk_pr_mean_da = uk_mon_pr_ds["pr"].isel(ensemble_member=0).mean(dim="time")
london_pr_mean_da = SelectDomain(subdomain="london").run(uk_pr_mean_da).isel(grid_latitude=slice(32), grid_longitude=slice(32))

In [ ]:
fig, axd = plt.subplot_mosaic([["uk", "london"]], figsize=(14, 6), subplot_kw={'projection': cp_model_rotated_pole}, constrained_layout=True)

ax = axd["uk"]
plot_grid(uk_pr_mean_da, ax=ax, title="UK Mean Precip [mm/day]", transform=cp_model_rotated_pole, cmap="Blues", norm=None, add_colorbar=True)

ax = axd["london"]
plot_grid(london_pr_mean_da, ax=ax, title="London area Mean Precip [mm/day]", transform=cp_model_rotated_pole, cmap="Blues", norm=None, add_colorbar=True)

plt.show()

## Compare training set properties

In [ ]:
dataset_name = "2.2km-coarsened-8x_london_vorticity850_random"
train_ds = xr.open_dataset(os.path.join(os.getenv("MOOSE_DERIVED_DATA"), "nc-datasets", dataset_name, "train.nc")).isel(grid_latitude=slice(32), grid_longitude=slice(32))
val_ds = xr.open_dataset(os.path.join(os.getenv("MOOSE_DERIVED_DATA"), "nc-datasets", dataset_name, "val.nc")).isel(grid_latitude=slice(32), grid_longitude=slice(32))

val_ds["target_pr"] = (val_ds["target_pr"]*3600*24).assign_attrs({"units": "mm day-1"})
train_ds["target_pr"] = (train_ds["target_pr"]*3600*24).assign_attrs({"units": "mm day-1"})

train_pr_mean_da = train_ds["target_pr"].mean(dim="time")
val_pr_mean_da = val_ds["target_pr"].mean(dim="time")

### Means

In [ ]:
fig, axd = plt.subplot_mosaic([["Full", "Train", "Val"]], figsize=(18, 6), subplot_kw=dict(projection=cp_model_rotated_pole), constrained_layout=True)

vmin = min([da.min().values for da in [train_pr_mean_da, val_pr_mean_da, london_pr_mean_da]])
vmax = max([da.max().values for da in [train_pr_mean_da, val_pr_mean_da, london_pr_mean_da]])

ax = axd["Full"]
plot_grid(london_pr_mean_da, ax, title="Full pr mean", norm=matplotlib.colors.Normalize(vmin=vmin, vmax=vmax), add_colorbar=False)

ax = axd["Train"]
plot_grid(train_pr_mean_da, ax, title="Training set pr mean", norm=None, vmin=vmin, vmax=vmax, add_colorbar=True)

ax = axd["Val"]
plot_grid(val_pr_mean_da, ax, title="Validation set pr mean", norm=None, vmin=vmin, vmax=vmax, add_colorbar=False)

plt.show()

fig, axd = plt.subplot_mosaic([["Full - Train Mean", "Full - Train Mean Ratio"], ["Full - Val Mean", "Full - Val Mean Ratio"]], figsize=(12, 12), subplot_kw=dict(projection=cp_model_rotated_pole), constrained_layout=True)

ax = axd["Full - Train Mean"]
plot_grid((london_pr_mean_da - train_pr_mean_da), ax, title="Full - Train Mean", norm=None, add_colorbar=True, cmap='BrBG', center=0)

ax = axd["Full - Train Mean Ratio"]
plot_grid(((london_pr_mean_da - train_pr_mean_da)/london_pr_mean_da), ax, title="Full - Train Mean Ratio", norm=None, add_colorbar=True, cmap='BrBG', center=0)

ax = axd["Full - Val Mean"]
plot_grid((london_pr_mean_da - val_pr_mean_da), ax, title="Full - Val Mean", norm=None, add_colorbar=True, cmap='BrBG', center=0)

ax = axd["Full - Val Mean Ratio"]
plot_grid(((london_pr_mean_da - val_pr_mean_da)/london_pr_mean_da), ax, title="Full - Val Mean Ratio", norm=None, add_colorbar=True, cmap='BrBG', center=0)

plt.show()